# Simulator Examples

In this notebook we try to use the MDP simulator of PGCL programs and develop a notion of programs as transformers of state distributions. From this we can hopefully derive some notion of expectation.  We begin with loading the padlock jar:

In [9]:
interp.load.cp (os.pwd/"padlock-0.0.jar")

The `pgcl` package contains the abstract syntax types for PGCL.  The `pgcl.syntax` package contains extension methods that allow to create the abstract syntax of PGCL models in a stylized internal DSL. Not very pretty, but easier to handle than the AST directly.  The `mdp` package contains the implementation of the MDP semantics (a small-step continuation passing style reducer for programs and a small-step continuation passing style evaluator for expressions), the types of runtime values, and the types of schedulers able to resolve both probabilistic and demonic choices.

In [10]:
import padlock.pgcl._
import padlock.pgcl.syntax._
import padlock.mdp._

import padlock.pgcl._

import padlock.pgcl.syntax._

import padlock.mdp._

We create several examples to have something to experiment with:

In [12]:
val coin = "coin"
val head = True
val tail = False

val example0 = coin := head

val example1 = coin := tail

val example2 = (example0) < 0.5 > (example1)

val example3 = (
    (coin := tail)
    ((coin := head) < 0.5 > (coin := tail))
)

val example4 = (
    ((coin := head) < 0.5 > (coin := tail))
    (coin := head)
)

coin: String = "coin"
head: True.type = True
tail: False.type = False
example0: Assgn = Assgn("coin", ValExpr(True))
example1: Assgn = Assgn("coin", ValExpr(False))
example2: Probabilistic = Probabilistic(
  Assgn("coin", ValExpr(True)),
  ValExpr(ValP(0.5)),
  Assgn("coin", ValExpr(False))
)
example3: Seq = Seq(
  Assgn("coin", ValExpr(False)),
  Probabilistic(
    Assgn("coin", ValExpr(True)),
    ValExpr(ValP(0.5)),
    Assgn("coin", ValExpr(False))
  )
)
example4: Seq = Seq(
  Probabilistic(
    Assgn("coin", ValExpr(True)),
    ValExpr(ValP(0.5)),
    Assgn("coin", ValExpr(False))
  ),
  Assgn("coin", ValExpr(True))
)

We can run an example with a default scheduler initialized with a constant seed.   The `MDP` object encapsulates the simulator.  The `run1` method reduces the provided program until it terminates or aborts (or otherwise the simulator fails, possibly due to bugs).  The method takes up to three arguments, but we are using the simples non-probabilistic version.  Each run will give you the same arbitrary outcome! It is called `run1` because it does not produce a sample set of runs, but only a single result.

In [13]:
MDP.run1 (example2)

res12: Either[String, Env] = Right(Map("coin" -> RuntimeValB(false)))

The `SimpleScheduler` companion object defines several fixed schedulers that we can use to change the behaviour of `run1`. Below I instantiate a scheduler that always chooses a left branch in a non-deterministic choice.  The seed 42 is used to initialize the pseudo random number generator state of the probabilistic part of the scheduler.  The second scheduler (`fairCoin`) will be a scheduler that randomly (with probability 0.5) will resolve every demonic choice.  It is initialized with the same seed. 

In [14]:
val alwaysLeft = SimpleScheduler.AlwaysLeftScheduler[Env] (42)
val fairCoin = SimpleScheduler.FairCoinScheduler[Env] (42)

alwaysLeft: SimpleScheduler[Env] = padlock.mdp.SimpleScheduler@64a159a2
fairCoin: SimpleScheduler[Env] = padlock.mdp.SimpleScheduler$$anon$1@74cce441

Now we can use one of the schedulers to run our example.  For `example0` and `example1` all schedulers should behave the same as there are no probabilistic nor demonic choices involved.  For so simple examples, it is thus better to only use one argument to the function `run1`, which assumes some default scheduler. For `example2` the choice of the scheduler does not make any difference either, because all padlock schedulers (so far) behave exactly the same for probabilistic choices.  They differ only on demonic choices. But for an example with a probabilistic choice (`example2`) the choice of the initial seed does make a difference.

In [15]:
MDP.run1 (example0, alwaysLeft)
MDP.run1 (example0)

res14_0: Either[String, Env] = Right(Map("coin" -> RuntimeValB(true)))
res14_1: Either[String, Env] = Right(Map("coin" -> RuntimeValB(true)))

In [19]:
MDP.run1 (example1, alwaysLeft)
MDP.run1 (example1, fairCoin)

res18_0: Either[String, Env] = Right(Map("coin" -> RuntimeValB(false)))
res18_1: Either[String, Env] = Right(Map("coin" -> RuntimeValB(false)))

In [24]:
MDP.run1 (example2, alwaysLeft)
MDP.run1 (example2, SimpleScheduler.AlwaysLeftScheduler[Env] (1))

res23_0: Either[String, Env] = Right(Map("coin" -> RuntimeValB(false)))
res23_1: Either[String, Env] = Right(Map("coin" -> RuntimeValB(false)))

In [8]:
MDP.run1 (example3, alwaysLeft)

res7: Either[String, Env] = Right(Map("coin" -> RuntimeValB(false)))

In [12]:
MDP.run1 (example4, alwaysLeft)

res11: Either[String, Env] = Right(Map("coin" -> RuntimeValB(true)))

What is an expectation?

If we have a prior probability distribution $\pi$ on the variable _coin_ then we would like to know what is the probability on the distribution after. So we want to compute something like:

sample env $\in \pi$; collect the values of MDP.run1 (example4, faircoin, env)

Then plot the histogram of the values obtained.  But this is still not using Bayesian inference. 

With Figaro I would've been able to represent \pi and map the reducer to obtain the final value.  

TODO:
* Figure out how to infer distributions (We need to start revisiting the various distribution frameworks)
* Figure out how to plot distributions
* Mark expectations for the distributions
* For this to work in proofs we must be able to perform bayesian decision making on small pieces of programs, possibly automatically  (is ROPE overlapping with HDI)? This I might choose to work on much later.

## Defining Schedulers

In [ ]:
TODO: Show different ways of defining schedulers.